In [2]:
import torch
import pandas as pd
import numpy as np
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer

from model.bi_lstm import BiLSTMAttentionABSA
from model.pho_bert import PhoBERT_ABSA

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [3]:
class ABSATestDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_len=128):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.aspects = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        cleaned_review = str(self.data.iloc[index]['Cleaned_Review'])
        
        if 'Review' in self.data.columns:
            original_review = str(self.data.iloc[index]['Review'])
        else:
            original_review = cleaned_review
            
        labels = []
        for aspect in self.aspects:
            label = self.data.iloc[index][aspect]
            labels.append(3 if label == -1 else label)
            
        encoding = self.tokenizer(
            cleaned_review,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'original_review': original_review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [4]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

test_dataset = ABSATestDataset('datasets_cleaned/test_data.csv', tokenizer)

print("Đang nạp trọng số BiLSTM-Attention...")
bilstm_model = BiLSTMAttentionABSA(vocab_size=64001, embedding_dim=256, hidden_dim=256).to(device)
bilstm_model.load_state_dict(torch.load('saved_models/bilstm_absa_weights.pth', map_location=device))
bilstm_model.eval()
print("\nSuccess!")

print("Đang nạp trọng số PhoBERT...")
phobert_model = PhoBERT_ABSA().to(device)
phobert_model.load_state_dict(torch.load('saved_models/phobert_absa_weights.pth', map_location=device))
phobert_model.eval()
print("\nSuccess!")

Đang nạp trọng số BiLSTM-Attention...

Success!
Đang nạp trọng số PhoBERT...


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 627.76it/s, Materializing param=pooler.dense.weight]                               
RobertaModel LOAD REPORT from: vinai/phobert-base
Key                             | Status     |  | 
--------------------------------+------------+--+-
lm_head.dense.weight            | UNEXPECTED |  | 
lm_head.layer_norm.weight       | UNEXPECTED |  | 
lm_head.bias                    | UNEXPECTED |  | 
roberta.embeddings.position_ids | UNEXPECTED |  | 
lm_head.layer_norm.bias         | UNEXPECTED |  | 
lm_head.dense.bias              | UNEXPECTED |  | 
lm_head.decoder.weight          | UNEXPECTED |  | 
lm_head.decoder.bias            | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Success!


In [5]:
aspects = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']
label_map = {0: 'Tiêu cực 🔴', 1: 'Tích cực 🟢', 2: 'Trung tính 🟡', 3: 'Không đề cập ⚪'}

def compare_on_test_sample(sample_idx):
    sample = test_dataset[sample_idx]
    review_text = sample['original_review']
    
    print(f"{'='*100}")
    print(f"📝 Câu gốc [{sample_idx}]: '{review_text}'")
    print(f"{'-'*100}")
    
    input_ids = sample['input_ids'].unsqueeze(0).to(device)
    attention_mask = sample['attention_mask'].unsqueeze(0).to(device)
    true_labels = sample['labels'].numpy()
    
    with torch.no_grad():
        bilstm_outputs = bilstm_model(input_ids)
        phobert_outputs = phobert_model(input_ids=input_ids, attention_mask=attention_mask)
        
    print(f"{'Khía cạnh':<15} | {'Thực tế (Ground Truth)':<22} | {'BiLSTM Dự đoán':<25} | {'PhoBERT Dự đoán':<25}")
    print("-" * 100)
    
    found_any = False
    for i, aspect in enumerate(aspects):
        true_l = true_labels[i]
        bilstm_pred = torch.argmax(bilstm_outputs[i], dim=1).item()
        phobert_pred = torch.argmax(phobert_outputs[i], dim=1).item()
        
        if true_l != 3 or bilstm_pred != 3 or phobert_pred != 3:
            true_str = label_map[true_l]
            bilstm_str = label_map[bilstm_pred]
            phobert_str = label_map[phobert_pred]
            
            bilstm_mark = "✅" if bilstm_pred == true_l else "❌"
            phobert_mark = "✅" if phobert_pred == true_l else "❌"
            
            print(f"{aspect:<15} | {true_str:<22} | {bilstm_str} {bilstm_mark:<8} | {phobert_str} {phobert_mark}")
            found_any = True
            
    if not found_any:
        print("(Câu này hoàn toàn không chứa đánh giá khen/chê nào)")
    print(f"{'='*100}\n")

In [7]:
num_samples = 20

random.seed(42) 
random_indices = random.sample(range(len(test_dataset)), num_samples)

print("🚀 BẮT ĐẦU SO SÁNH HAI MÔ HÌNH TRÊN TẬP TEST THỰC TẾ:\n")
for idx in random_indices:
    compare_on_test_sample(idx)

🚀 BẮT ĐẦU SO SÁNH HAI MÔ HÌNH TRÊN TẬP TEST THỰC TẾ:

📝 Câu gốc [456]: 'Ngoài việc màu bị lem ra đế còn lại thì oke'
----------------------------------------------------------------------------------------------------
Khía cạnh       | Thực tế (Ground Truth) | BiLSTM Dự đoán            | PhoBERT Dự đoán          
----------------------------------------------------------------------------------------------------
Outlook         | Tiêu cực 🔴             | Tiêu cực 🔴 ✅        | Tiêu cực 🔴 ✅
General         | Không đề cập ⚪         | Trung tính 🟡 ❌        | Trung tính 🟡 ❌

📝 Câu gốc [102]: 'Giày mới thì mang vài thấy hơi chật thôi mang nhiều giày sẽ giãn ra theo chân là oke ko cần mua size to hơn làm j'
----------------------------------------------------------------------------------------------------
Khía cạnh       | Thực tế (Ground Truth) | BiLSTM Dự đoán            | PhoBERT Dự đoán          
--------------------------------------------------------------------------------------------